The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:

## Task 1: Mark all launch sites on a map



First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.

In [3]:
import aiohttp
print("aiohttp is installed successfully!")
import asyncio
print("asyncio is installed successfully!")

aiohttp is installed successfully!
asyncio is installed successfully!


In [4]:
async def download(url, filename):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            if response.status == 200:
                with open(filename, "wb") as f:
                    f.write(await response.read())
                print(f"File downloaded successfully: {filename}")
            else:
                print(f"Failed to download file. HTTP Status: {response.status}")

# Define the URL and filename
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

filename = "spacex_csv_file"

# Call the coroutine directly with await (in environments with a running loop, like Jupyter)
await download(url, filename)

File downloaded successfully: spacex_csv_file


In [5]:
spacex_df = pd.read_csv(filename)

In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,

In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame `launch_sites`

Now, let's add a circle for each launch site in data frame `launch_sites`

An example of folium.Circle:

`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`

An example of folium.Marker:

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`

In [13]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# add a Circle object based on its coordinate (Lat, Long) values for each launch site. In addition, add Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    folium.Circle(
        location=coordinate,
        radius=8000,
        color='#d35400',
        fill=True,
        fill_color='#3186cc',
        popup=row['Launch Site']
    ).add_to(site_map)

# Add a marker with a label
    folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)


site_map

The generated map with marked launch sites should look similar to the following:


Now, you can explore the map by zoom-in/out the marked areas , and try to answer the following questions:

Are all launch sites in proximity to the Equator line?
The Equator is at 0° latitude. Launch sites close to the Equator benefit from Earth's rotational speed, which provides an extra velocity boost for rockets. Let's analyze the latitudes of the SpaceX launch sites:

From the launch_sites_df data, we can observe the latitudes:
Kennedy Space Center (KSC): ~28.6°N
Cape Canaveral Space Force Station (CCSFS): ~28.5°N
Vandenberg Space Force Base (VSFB): ~34.7°N
Boca Chica (SpaceX South Texas): ~25.9°N
Findings:

All the SpaceX launch sites are relatively close to the Equator compared to other global launch sites (e.g., Baikonur Cosmodrome at ~46°N or Plesetsk at ~62°N).
However, they are not extremely close to the Equator. The closest site (Boca Chica) is still about 26° north of the Equator.

Are all launch sites in very close proximity to the coast?

Launch sites are often located near coastlines to ensure:

Safety: Rockets can be launched over the ocean, minimizing risk to populated areas in case of failure.
Efficiency: Coastal locations allow for polar and equatorial orbit trajectories without crossing landmasses.
Also please try to explain your findings.

Analysis of the SpaceX launch sites:

Kennedy Space Center (KSC): Close to the Atlantic Ocean.
Cape Canaveral Space Force Station (CCSFS): Adjacent to the Atlantic Ocean.
Vandenberg Space Force Base (VSFB): Located along the Pacific Ocean.
Boca Chica (SpaceX South Texas): Right by the Gulf of Mexico.
Findings:

All SpaceX launch sites are very close to coastlines, which aligns with standard safety and operational considerations for launch facilities.


# Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not

In [16]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a MarkerCluster object

In [17]:
marker_cluster = MarkerCluster()

TODO: Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value

In [18]:
import numpy as np

In [30]:

print("Successes:", len(spacex_df[spacex_df['class'] == 1]))
print("Failures:", len(spacex_df[spacex_df['class'] == 0]))

# Initialize the map
nasa_coordinate = [29.563197, -95.576820]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a MarkerCluster to the map
marker_cluster = MarkerCluster().add_to(site_map)



# Loop through the DataFrame and add a CircleMarker for each launch
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_status = row['class']  # 1 for success, 0 for failure
    
    # Define marker color based on launch status
    marker_color = 'green' if launch_status == 1 else 'red'
    
    # Define popup label
    popup_label = f"Launch Site: {row['Launch Site']}<br>Status: {'Success' if launch_status == 1 else 'Failure'}"
    
    # Add the CircleMarker to the map
    folium.Marker(
        location=coordinate,
        radius=3,
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=0.6,
        popup=popup_label,
        icon=folium.Icon(color=marker_color)
    ).add_to(marker_cluster)

site_map


Successes: 24
Failures: 32


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

# TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [31]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [32]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [54]:
from math import sin, cos, sqrt, atan2, radians
import folium
from folium.features import DivIcon
from folium.plugins import MousePosition

# Function to calculate distance between two points (Haversine formula)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of Earth in km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Example coordinates for the launch site and other points
launch_site_lat = 28.562302
launch_site_lon = -80.577356
coastline_lat = 28.563857
coastline_lon = -80.567955
highway_lat = 28.56321
highway_lon = -80.5708
railway_lat = 28.56381
railway_lon = -80.58693
city_lat = 28.07433
city_lon = -80.61704

# Calculate distances
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Initialize the map
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

# Define features and distances
features = [
    ('Closest Coastline', coastline_lat, coastline_lon, distance_coastline, 'green'),
    ('Highway', highway_lat, highway_lon, distance_highway, 'purple'),
    ('Railway', railway_lat, railway_lon, distance_railway, 'orange'),
    ('Closest City', city_lat, city_lon, distance_city, 'beige')
]

# Add markers, distance labels, and polylines for each feature
for name, lat, lon, distance, color in features:
    # Add the feature marker
    folium.Marker(
        [lat, lon],
        popup=name,
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(site_map)
    
    # Add a distance marker
    folium.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance),
        )
    ).add_to(site_map)
    
    # Add a PolyLine from the launch site to the feature
    coordinates = [[launch_site_lat, launch_site_lon], [lat, lon]]
    folium.PolyLine(locations=coordinates, weight=2, color=color).add_to(site_map)

# Add MousePosition to view coordinates dynamically
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Display the map
site_map
